# Data Exploration

## Import File and Set Parameters

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# enter desired years of data
YEARS = [2020,2019,2018,2017,2016,2015,2014,2013,2012,2011]

data = pd.DataFrame()

for i in YEARS:  
    # low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    # sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

# give each row a unique index
data.reset_index(drop=True, inplace=True)

data.head()

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,...,xyac_median_yardage,xyac_success,yac_epa,yac_wpa,yardline_100,yards_after_catch,yards_gained,ydsnet,ydstogo,yrdln
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,ARI 35
1,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,35.0,NaN,0.0,41.0,0,ARI 35
2,0,-0.132787,0.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,...,2.0,0.619306,1.427625,0.033715,75.0,1.0,5.0,41.0,10,SF 25
3,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,55.0,NaN,14.0,41.0,10,SF 45
4,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,41.0,NaN,2.0,41.0,10,ARI 41


In [2]:
# model only focuses offensive 4th down plays
data['down'].value_counts()

1.0    162396
2.0    121007
3.0     77083
4.0     41145
Name: down, dtype: int64

In [3]:
data = data[data['down'] == 4.0]
data['down'].value_counts()

4.0    41145
Name: down, dtype: int64

In [4]:
# a special teams play converstion that is not a punt or kick could be considered a fake punt or kick
# these plays will not be considered conversion attempts for this model

data['special_teams_play'].value_counts()

1    35490
0     5655
Name: special_teams_play, dtype: int64

In [5]:
data = data[data['special_teams_play'] == 0]
data['special_teams_play'].value_counts()

0    5655
Name: special_teams_play, dtype: int64

In [6]:
data['play_type'].value_counts()

pass        3281
run         1685
no_play      640
qb_kneel      33
punt           3
Name: play_type, dtype: int64

In [7]:
# no play means a penalty and the conversion is tried again
# qb_kneel is to run the clock out and is not considered a conversion attempt in this model
# qb_spike is to stop the clock and is not considered a conversion attempt in this model
# offenses will fake a conversation and their qb will punt
# the model is not concerned with these plays

data = data[~(data['play_type'].isin(['no_play', 'qb_kneel', 'qb_spike', 'punt']))]
data['play_type'].value_counts()

pass    3281
run     1685
Name: play_type, dtype: int64

In [8]:
data['season_type'].value_counts()

REG     4727
POST     252
Name: season_type, dtype: int64

In [9]:
data = data[data['season_type'] == 'REG']
data['season_type'].value_counts()

REG    4727
Name: season_type, dtype: int64

## Drop Useless Columns

In [10]:
list(data)

['aborted_play',
 'air_epa',
 'air_wpa',
 'air_yards',
 'assist_tackle',
 'assist_tackle_1_player_id',
 'assist_tackle_1_player_name',
 'assist_tackle_1_team',
 'assist_tackle_2_player_id',
 'assist_tackle_2_player_name',
 'assist_tackle_2_team',
 'assist_tackle_3_player_id',
 'assist_tackle_3_player_name',
 'assist_tackle_3_team',
 'assist_tackle_4_player_id',
 'assist_tackle_4_player_name',
 'assist_tackle_4_team',
 'away_coach',
 'away_score',
 'away_team',
 'away_timeouts_remaining',
 'away_wp',
 'away_wp_post',
 'blocked_player_id',
 'blocked_player_name',
 'comp_air_epa',
 'comp_air_wpa',
 'comp_yac_epa',
 'comp_yac_wpa',
 'complete_pass',
 'cp',
 'cpoe',
 'def_wp',
 'defensive_extra_point_attempt',
 'defensive_extra_point_conv',
 'defensive_two_point_attempt',
 'defensive_two_point_conv',
 'defteam',
 'defteam_score',
 'defteam_score_post',
 'defteam_timeouts_remaining',
 'desc',
 'div_game',
 'down',
 'drive',
 'drive_end_transition',
 'drive_end_yard_line',
 'drive_ended_with_

In [11]:
df = data[['season', 'week', 'home_team', 'away_team', 'posteam_type', 'posteam', 'defteam', 'yardline_100', 
           'half_seconds_remaining', 'game_half', 'drive', 'qtr', 'down', 'goal_to_go', 'ydstogo', 'ydsnet', 'yards_gained', 
           'shotgun', 'no_huddle', 'qb_dropback', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 
           'yards_after_catch', 'run_location', 'run_gap', 'timeout', 'timeout_team', 'posteam_timeouts_remaining', 
           'defteam_timeouts_remaining', 'score_differential', 'score_differential_post', 'no_score_prob', 'opp_td_prob', 
           'td_prob', 'ep', 'epa', 'wp', 'wpa', 'complete_pass', 'incomplete_pass', 'interception', 'tackled_for_loss', 
           'fumble_lost', 'qb_hit', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'fumble', 'series', 
           'drive_play_count', 'drive_inside20', 'div_game', 'stadium', 'surface', 'roof', 'pass', 'rush', 'success']].copy()

In [12]:
df.head(50)

,season,week,home_team,away_team,posteam_type,posteam,defteam,yardline_100,half_seconds_remaining,game_half,...,series,drive_play_count,drive_inside20,div_game,stadium,surface,roof,pass,rush,success
49,2020,1,SF,ARI,home,SF,ARI,1.0,757.0,Half1,...,16,10.0,1.0,1,Levi's Stadium,grass,outdoors,0.0,1.0,0.0
181,2020,1,SF,ARI,home,SF,ARI,16.0,37.0,Half2,...,62,12.0,1.0,1,Levi's Stadium,grass,outdoors,1.0,0.0,0.0
208,2020,1,DET,CHI,away,CHI,DET,34.0,1442.0,Half1,...,6,8.0,0.0,1,Ford Field,fieldturf,dome,1.0,0.0,0.0
323,2020,1,DET,CHI,away,CHI,DET,10.0,900.0,Half2,...,49,11.0,1.0,1,Ford Field,fieldturf,dome,0.0,1.0,1.0
511,2020,1,BAL,CLE,away,CLE,BAL,28.0,1088.0,Half2,...,46,10.0,0.0,1,M&T Bank Stadium,grass,outdoors,1.0,0.0,0.0
533,2020,1,BAL,CLE,away,CLE,BAL,48.0,611.0,Half2,...,54,7.0,0.0,1,M&T Bank Stadium,grass,outdoors,1.0,0.0,0.0
584,2020,1,LA,DAL,home,LA,DAL,41.0,1214.0,Half1,...,10,17.0,1.0,0,SoFi Stadium,matrixturf,dome,0.0,1.0,1.0
708,2020,1,LA,DAL,away,DAL,LA,11.0,706.0,Half2,...,54,9.0,1.0,0,SoFi Stadium,matrixturf,dome,1.0,0.0,0.0
737,2020,1,LA,DAL,away,DAL,LA,70.0,56.0,Half2,...,62,10.0,0.0,0,SoFi Stadium,matrixturf,dome,0.0,1.0,1.0
743,2020,1,LA,DAL,away,DAL,LA,76.0,13.0,Half2,...,63,10.0,0.0,0,SoFi Stadium,matrixturf,dome,1.0,0.0,0.0


In [13]:
df.describe()

,season,week,yardline_100,half_seconds_remaining,drive,qtr,down,goal_to_go,ydstogo,ydsnet,...,pass_touchdown,rush_touchdown,fumble,series,drive_play_count,drive_inside20,div_game,pass,rush,success
count,4727.000000,4727.000000,4727.000000,4727.000000,4727.000000,4727.00000,4727.0,4727.000000,4727.000000,4727.000000,...,4716.000000,4716.000000,4716.000000,4727.000000,4727.000000,4727.000000,4727.000000,4716.000000,4727.000000,4713.000000
mean,2015.872858,9.408081,35.793738,545.254072,15.901417,3.22234,4.0,0.103871,4.472181,47.113603,...,0.056616,0.031807,0.026718,40.487413,9.341654,0.474297,0.386715,0.686599,0.312672,0.496923
std,2.921261,4.989363,21.631194,489.389464,7.147234,1.06271,0.0,0.305126,4.789653,25.669363,...,0.231131,0.175504,0.161274,17.447709,3.516999,0.499392,0.487049,0.463925,0.463631,0.500044
min,2011.000000,1.000000,1.000000,0.000000,1.000000,1.00000,4.0,0.000000,1.000000,-30.000000,...,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.000000,5.000000,19.000000,120.000000,11.000000,2.00000,4.0,0.000000,1.000000,27.000000,...,0.000000,0.000000,0.000000,27.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2016.000000,10.000000,36.000000,385.000000,17.000000,4.00000,4.0,0.000000,2.000000,49.000000,...,0.000000,0.000000,0.000000,46.000000,9.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,2019.000000,14.000000,50.000000,900.000000,21.000000,4.00000,4.0,0.000000,6.000000,70.000000,...,0.000000,0.000000,0.000000,54.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,2020.000000,17.000000,97.000000,1800.000000,34.000000,5.00000,4.0,1.000000,43.000000,99.000000,...,1.000000,1.000000,1.000000,83.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
df['posteam'].value_counts()

PHI    211
CLE    203
JAX    187
NYG    174
BAL    163
CHI    161
ATL    158
GB     157
CIN    154
WAS    154
HOU    151
IND    150
MIA    148
MIN    148
NYJ    145
LV     145
CAR    142
BUF    142
TB     140
SF     140
ARI    138
NO     137
DAL    137
DET    137
LA     136
NE     136
DEN    135
TEN    129
KC     121
LAC    119
SEA    116
PIT    113
Name: posteam, dtype: int64

In [15]:
df['defteam'].value_counts()

KC     188
NE     188
MIN    184
GB     182
NO     170
CHI    168
BAL    167
SF     163
TEN    161
PIT    156
CIN    151
ATL    151
DAL    150
MIA    148
NYG    148
DEN    147
LA     147
PHI    147
WAS    145
CLE    143
CAR    143
BUF    135
ARI    135
HOU    133
SEA    130
TB     129
DET    128
IND    123
NYJ    122
LAC    121
LV     112
JAX    112
Name: defteam, dtype: int64

## Feature Engineer Offensive and Defensive Rankings For Each Season

### Using data from The Football Database, individual rankings will be assigned to offensive and defensive teams for every season

In [16]:
def off_rank(i):
    if i['season'] == 2011:
        if i['posteam'] == 'NO':
            val = 1
        elif i['posteam'] == 'NE':
            val = 2
        elif i['posteam'] == 'GB':
            val = 3
        elif i['posteam'] == 'PHI':
            val = 4
        elif i['posteam'] == 'DET':
            val = 5
        elif i['posteam'] == 'LAC':
            val = 6
        elif i['posteam'] == 'CAR':
            val = 7
        elif i['posteam'] == 'NYG':
            val = 8
        elif i['posteam'] == 'LV':
            val = 9
        elif i['posteam'] == 'ATL':
            val = 10
        elif i['posteam'] == 'DAL':
            val = 11
        elif i['posteam'] == 'PIT':
            val = 12
        elif i['posteam'] == 'HOU':
            val = 13
        elif i['posteam'] == 'BUF':
            val = 14
        elif i['posteam'] == 'BAL':
            val = 15
        elif i['posteam'] == 'WAS':
            val = 16
        elif i['posteam'] == 'TEN':
            val = 17
        elif i['posteam'] == 'MIN':
            val = 18
        elif i['posteam'] == 'ARI':
            val = 19
        elif i['posteam'] == 'CIN':
            val = 20
        elif i['posteam'] == 'TB':
            val = 21
        elif i['posteam'] == 'MIA':
            val = 22
        elif i['posteam'] == 'DEN':
            val = 23
        elif i['posteam'] == 'CHI':
            val = 24
        elif i['posteam'] == 'NYJ':
            val = 25
        elif i['posteam'] == 'SF':
            val = 26
        elif i['posteam'] == 'KC':
            val = 27
        elif i['posteam'] == 'SEA':
            val = 28
        elif i['posteam'] == 'CLE':
            val = 29
        elif i['posteam'] == 'IND':
            val = 30
        elif i['posteam'] == 'LAR':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2012:
        if i['posteam'] == 'NE':
            val = 1
        elif i['posteam'] == 'NO':
            val = 2
        elif i['posteam'] == 'DET':
            val = 3
        elif i['posteam'] == 'DEN':
            val = 4
        elif i['posteam'] == 'WAS':
            val = 5
        elif i['posteam'] == 'DAL':
            val = 6
        elif i['posteam'] == 'HOU':
            val = 7
        elif i['posteam'] == 'ATL':
            val = 8
        elif i['posteam'] == 'TB':
            val = 9
        elif i['posteam'] == 'IND':
            val = 10
        elif i['posteam'] == 'SF':
            val = 11
        elif i['posteam'] == 'CAR':
            val = 12
        elif i['posteam'] == 'GB':
            val = 13
        elif i['posteam'] == 'NYG':
            val = 14
        elif i['posteam'] == 'PHI':
            val = 15
        elif i['posteam'] == 'BAL':
            val = 16
        elif i['posteam'] == 'SEA':
            val = 17
        elif i['posteam'] == 'LV':
            val = 18
        elif i['posteam'] == 'BUF':
            val = 19
        elif i['posteam'] == 'MIN':
            val = 20
        elif i['posteam'] == 'PIT':
            val = 21
        elif i['posteam'] == 'CIN':
            val = 22
        elif i['posteam'] == 'LAR':
            val = 23
        elif i['posteam'] == 'KC':
            val = 24
        elif i['posteam'] == 'CLE':
            val = 25
        elif i['posteam'] == 'TEN':
            val = 26
        elif i['posteam'] == 'MIA':
            val = 27
        elif i['posteam'] == 'CHI':
            val = 28
        elif i['posteam'] == 'JAX':
            val = 29
        elif i['posteam'] == 'NYJ':
            val = 30
        elif i['posteam'] == 'LAC':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2013:
        if i['posteam'] == 'DEN':
            val = 1
        elif i['posteam'] == 'PHI':
            val = 2
        elif i['posteam'] == 'GB':
            val = 3
        elif i['posteam'] == 'NO':
            val = 4
        elif i['posteam'] == 'LAC':
            val = 5
        elif i['posteam'] == 'DET':
            val = 6
        elif i['posteam'] == 'NE':
            val = 7
        elif i['posteam'] == 'CHI':
            val = 8
        elif i['posteam'] == 'WAS':
            val = 9
        elif i['posteam'] == 'CIN':
            val = 10
        elif i['posteam'] == 'HOU':
            val = 11
        elif i['posteam'] == 'ARI':
            val = 12
        elif i['posteam'] == 'MIN':
            val = 13
        elif i['posteam'] == 'ATL':
            val = 14
        elif i['posteam'] == 'IND':
            val = 15
        elif i['posteam'] == 'DAL':
            val = 16
        elif i['posteam'] == 'CLE':
            val = 17
        elif i['posteam'] == 'SEA':
            val = 18
        elif i['posteam'] == 'BUF':
            val = 19
        elif i['posteam'] == 'PIT':
            val = 20
        elif i['posteam'] == 'KC':
            val = 21
        elif i['posteam'] == 'TEN':
            val = 22
        elif i['posteam'] == 'LV':
            val = 23
        elif i['posteam'] == 'SF':
            val = 24
        elif i['posteam'] == 'NYJ':
            val = 25
        elif i['posteam'] == 'CAR':
            val = 26
        elif i['posteam'] == 'MIA':
            val = 27
        elif i['posteam'] == 'NYG':
            val = 28
        elif i['posteam'] == 'BAL':
            val = 29
        elif i['posteam'] == 'LAR':
            val = 30
        elif i['posteam'] == 'JAX':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2014:
        if i['posteam'] == 'NO':
            val = 1
        elif i['posteam'] == 'PIT':
            val = 2
        elif i['posteam'] == 'IND':
            val = 3
        elif i['posteam'] == 'DEN':
            val = 4
        elif i['posteam'] == 'PHI':
            val = 5
        elif i['posteam'] == 'GB':
            val = 6
        elif i['posteam'] == 'DAL':
            val = 7
        elif i['posteam'] == 'ATL':
            val = 8
        elif i['posteam'] == 'SEA':
            val = 9
        elif i['posteam'] == 'NYG':
            val = 10
        elif i['posteam'] == 'NE':
            val = 11
        elif i['posteam'] == 'BAL':
            val = 12
        elif i['posteam'] == 'WAS':
            val = 13
        elif i['posteam'] == 'MIA':
            val = 14
        elif i['posteam'] == 'CIN':
            val = 15
        elif i['posteam'] == 'CAR':
            val = 16
        elif i['posteam'] == 'HOU':
            val = 17
        elif i['posteam'] == 'LAC':
            val = 18
        elif i['posteam'] == 'DET':
            val = 19
        elif i['posteam'] == 'SF':
            val = 20
        elif i['posteam'] == 'CHI':
            val = 21
        elif i['posteam'] == 'NYJ':
            val = 22
        elif i['posteam'] == 'CLE':
            val = 23
        elif i['posteam'] == 'ARI':
            val = 24
        elif i['posteam'] == 'KC':
            val = 25
        elif i['posteam'] == 'BUF':
            val = 26
        elif i['posteam'] == 'MIN':
            val = 27
        elif i['posteam'] == 'LAR':
            val = 28
        elif i['posteam'] == 'TEN':
            val = 29
        elif i['posteam'] == 'TB':
            val = 30
        elif i['posteam'] == 'JAX':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2015:
        if i['posteam'] == 'ARI':
            val = 1
        elif i['posteam'] == 'NO':
            val = 2
        elif i['posteam'] == 'PIT':
            val = 3
        elif i['posteam'] == 'SEA':
            val = 4
        elif i['posteam'] == 'TB':
            val = 5
        elif i['posteam'] == 'NE':
            val = 6
        elif i['posteam'] == 'ATL':
            val = 7
        elif i['posteam'] == 'NYG':
            val = 8
        elif i['posteam'] == 'SEA':
            val = 9
        elif i['posteam'] == 'NYG':
            val = 10
        elif i['posteam'] == 'NE':
            val = 11
        elif i['posteam'] == 'BAL':
            val = 12
        elif i['posteam'] == 'WAS':
            val = 13
        elif i['posteam'] == 'MIA':
            val = 14
        elif i['posteam'] == 'CIN':
            val = 15
        elif i['posteam'] == 'CAR':
            val = 16
        elif i['posteam'] == 'HOU':
            val = 17
        elif i['posteam'] == 'LAC':
            val = 18
        elif i['posteam'] == 'DET':
            val = 19
        elif i['posteam'] == 'SF':
            val = 20
        elif i['posteam'] == 'CHI':
            val = 21
        elif i['posteam'] == 'NYJ':
            val = 22
        elif i['posteam'] == 'CLE':
            val = 23
        elif i['posteam'] == 'ARI':
            val = 24
        elif i['posteam'] == 'KC':
            val = 25
        elif i['posteam'] == 'BUF':
            val = 26
        elif i['posteam'] == 'MIN':
            val = 27
        elif i['posteam'] == 'LAR':
            val = 28
        elif i['posteam'] == 'TEN':
            val = 29
        elif i['posteam'] == 'TB':
            val = 30
        elif i['posteam'] == 'JAX':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2016:
        if i['posteam'] == 'NO':
            val = 1
        elif i['posteam'] == 'ATL':
            val = 2
        elif i['posteam'] == 'WAS':
            val = 3
        elif i['posteam'] == 'NE':
            val = 4
        elif i['posteam'] == 'DAL':
            val = 5
        elif i['posteam'] == 'LV':
            val = 6
        elif i['posteam'] == 'PIT':
            val = 7
        elif i['posteam'] == 'GB':
            val = 8
        elif i['posteam'] == 'ARI':
            val = 9
        elif i['posteam'] == 'IND':
            val = 10
        elif i['posteam'] == 'TEN':
            val = 11
        elif i['posteam'] == 'SEA':
            val = 12
        elif i['posteam'] == 'CIN':
            val = 13
        elif i['posteam'] == 'LAC':
            val = 14
        elif i['posteam'] == 'CHI':
            val = 15
        elif i['posteam'] == 'BUF':
            val = 16
        elif i['posteam'] == 'BAL':
            val = 17
        elif i['posteam'] == 'TB':
            val = 18
        elif i['posteam'] == 'CAR':
            val = 19
        elif i['posteam'] == 'KC':
            val = 20
        elif i['posteam'] == 'DET':
            val = 21
        elif i['posteam'] == 'PHI':
            val = 22
        elif i['posteam'] == 'JAX':
            val = 23
        elif i['posteam'] == 'MIA':
            val = 24
        elif i['posteam'] == 'NYG':
            val = 25
        elif i['posteam'] == 'NYJ':
            val = 26
        elif i['posteam'] == 'DEN':
            val = 27
        elif i['posteam'] == 'MIN':
            val = 28
        elif i['posteam'] == 'HOU':
            val = 29
        elif i['posteam'] == 'CLE':
            val = 30
        elif i['posteam'] == 'SF':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2017:
        if i['posteam'] == 'NE':
            val = 1
        elif i['posteam'] == 'NO':
            val = 2
        elif i['posteam'] == 'PIT':
            val = 3
        elif i['posteam'] == 'LAC':
            val = 4
        elif i['posteam'] == 'KC':
            val = 5
        elif i['posteam'] == 'JAX':
            val = 6
        elif i['posteam'] == 'PHI':
            val = 7
        elif i['posteam'] == 'ATL':
            val = 8
        elif i['posteam'] == 'TB':
            val = 9
        elif i['posteam'] == 'LAR':
            val = 10
        elif i['posteam'] == 'MIN':
            val = 11
        elif i['posteam'] == 'SF':
            val = 12
        elif i['posteam'] == 'DET':
            val = 13
        elif i['posteam'] == 'DAL':
            val = 14
        elif i['posteam'] == 'SEA':
            val = 15
        elif i['posteam'] == 'WAS':
            val = 16
        elif i['posteam'] == 'DEN':
            val = 17
        elif i['posteam'] == 'LV':
            val = 18
        elif i['posteam'] == 'CAR':
            val = 19
        elif i['posteam'] == 'HOU':
            val = 20
        elif i['posteam'] == 'NYG':
            val = 21
        elif i['posteam'] == 'ARI':
            val = 22
        elif i['posteam'] == 'TEN':
            val = 23
        elif i['posteam'] == 'CLE':
            val = 24
        elif i['posteam'] == 'MIA':
            val = 25
        elif i['posteam'] == 'GB':
            val = 26
        elif i['posteam'] == 'BAL':
            val = 27
        elif i['posteam'] == 'NYJ':
            val = 28
        elif i['posteam'] == 'BUF':
            val = 29
        elif i['posteam'] == 'CHI':
            val = 30
        elif i['posteam'] == 'IND':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2018:
        if i['posteam'] == 'KC':
            val = 1
        elif i['posteam'] == 'LAR':
            val = 2
        elif i['posteam'] == 'TB':
            val = 3
        elif i['posteam'] == 'PIT':
            val = 4
        elif i['posteam'] == 'NE':
            val = 5
        elif i['posteam'] == 'ATL':
            val = 6
        elif i['posteam'] == 'IND':
            val = 7
        elif i['posteam'] == 'NO':
            val = 8
        elif i['posteam'] == 'BAL':
            val = 9
        elif i['posteam'] == 'CAR':
            val = 10
        elif i['posteam'] == 'LAC':
            val = 11
        elif i['posteam'] == 'GB':
            val = 12
        elif i['posteam'] == 'CLE':
            val = 13
        elif i['posteam'] == 'PHI':
            val = 14
        elif i['posteam'] == 'HOU':
            val = 15
        elif i['posteam'] == 'SF':
            val = 16
        elif i['posteam'] == 'NYG':
            val = 17
        elif i['posteam'] == 'SEA':
            val = 18
        elif i['posteam'] == 'DEN':
            val = 19
        elif i['posteam'] == 'MIN':
            val = 20
        elif i['posteam'] == 'CHI':
            val = 21
        elif i['posteam'] == 'DAL':
            val = 22
        elif i['posteam'] == 'LV':
            val = 23
        elif i['posteam'] == 'DET':
            val = 24
        elif i['posteam'] == 'TEN':
            val = 25
        elif i['posteam'] == 'CIN':
            val = 26
        elif i['posteam'] == 'JAX':
            val = 27
        elif i['posteam'] == 'WAS':
            val = 28
        elif i['posteam'] == 'NYJ':
            val = 29
        elif i['posteam'] == 'BUF':
            val = 30
        elif i['posteam'] == 'MIA':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2019:
        if i['posteam'] == 'DAL':
            val = 1
        elif i['posteam'] == 'BAL':
            val = 2
        elif i['posteam'] == 'TB':
            val = 3
        elif i['posteam'] == 'SF':
            val = 4
        elif i['posteam'] == 'ATL':
            val = 5
        elif i['posteam'] == 'KC':
            val = 6
        elif i['posteam'] == 'LAR':
            val = 7
        elif i['posteam'] == 'SEA':
            val = 8
        elif i['posteam'] == 'NO':
            val = 9
        elif i['posteam'] == 'LAC':
            val = 10
        elif i['posteam'] == 'LV':
            val = 11
        elif i['posteam'] == 'TEN':
            val = 12
        elif i['posteam'] == 'HOU':
            val = 13
        elif i['posteam'] == 'PHI':
            val = 14
        elif i['posteam'] == 'NE':
            val = 15
        elif i['posteam'] == 'MIN':
            val = 16
        elif i['posteam'] == 'DET':
            val = 17
        elif i['posteam'] == 'GB':
            val = 18
        elif i['posteam'] == 'CAR':
            val = 19
        elif i['posteam'] == 'JAX':
            val = 20
        elif i['posteam'] == 'ARI':
            val = 21
        elif i['posteam'] == 'CLE':
            val = 22
        elif i['posteam'] == 'NYG':
            val = 23
        elif i['posteam'] == 'BUF':
            val = 24
        elif i['posteam'] == 'IND':
            val = 25
        elif i['posteam'] == 'CIN':
            val = 26
        elif i['posteam'] == 'MIA':
            val = 27
        elif i['posteam'] == 'DEN':
            val = 28
        elif i['posteam'] == 'CHI':
            val = 29
        elif i['posteam'] == 'PIT':
            val = 30
        elif i['posteam'] == 'WAS':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2020:
        if i['posteam'] == 'KC':
            val = 1
        elif i['posteam'] == 'BUF':
            val = 2
        elif i['posteam'] == 'TEN':
            val = 3
        elif i['posteam'] == 'MIN':
            val = 4
        elif i['posteam'] == 'GB':
            val = 5
        elif i['posteam'] == 'ARI':
            val = 6
        elif i['posteam'] == 'TB':
            val = 7
        elif i['posteam'] == 'LV':
            val = 8
        elif i['posteam'] == 'LAC':
            val = 9
        elif i['posteam'] == 'IND':
            val = 10
        elif i['posteam'] == 'LAR':
            val = 11
        elif i['posteam'] == 'NO':
            val = 12
        elif i['posteam'] == 'HOU':
            val = 13
        elif i['posteam'] == 'DAL':
            val = 14
        elif i['posteam'] == 'SF':
            val = 15
        elif i['posteam'] == 'CLE':
            val = 16
        elif i['posteam'] == 'SEA':
            val = 17
        elif i['posteam'] == 'ATL':
            val = 18
        elif i['posteam'] == 'BAL':
            val = 19
        elif i['posteam'] == 'DET':
            val = 20
        elif i['posteam'] == 'CAR':
            val = 21
        elif i['posteam'] == 'MIA':
            val = 22
        elif i['posteam'] == 'DEN':
            val = 23
        elif i['posteam'] == 'PHI':
            val = 24
        elif i['posteam'] == 'PIT':
            val = 25
        elif i['posteam'] == 'CHI':
            val = 26
        elif i['posteam'] == 'NE':
            val = 27
        elif i['posteam'] == 'JAX':
            val = 28
        elif i['posteam'] == 'CIN':
            val = 29
        elif i['posteam'] == 'WAS':
            val = 30
        elif i['posteam'] == 'NYG':
            val = 31
        else:
            val = 32
        return val
    else:
        return

In [17]:
df['off_rank'] = df.apply(off_rank, axis=1)

In [18]:
df['off_rank'].head(100)

49      15
181     15
208     26
323     26
511     16
        ..
7865    28
7992    10
8045    32
8146    31
8173    31
Name: off_rank, Length: 100, dtype: int64

In [19]:
df['off_rank'].unique()

array([15, 26, 16, 32, 14,  5,  4,  1, 13, 10,  9, 21, 27,  2, 24, 30, 31,
       25, 18, 17,  7, 12, 23,  3, 19, 22, 29, 20, 28,  8,  6, 11],
      dtype=int64)

In [20]:
def def_rank(i):
    if i['season'] == 2011:
        if i['defteam'] == 'PIT':
            val = 1
        elif i['defteam'] == 'HOU':
            val = 2
        elif i['defteam'] == 'BAL':
            val = 3
        elif i['defteam'] == 'SF':
            val = 4
        elif i['defteam'] == 'NYJ':
            val = 5
        elif i['defteam'] == 'JAX':
            val = 6
        elif i['defteam'] == 'CIN':
            val = 7
        elif i['defteam'] == 'PHI':
            val = 8
        elif i['defteam'] == 'SEA':
            val = 9
        elif i['defteam'] == 'CLE':
            val = 10
        elif i['defteam'] == 'KC':
            val = 11
        elif i['defteam'] == 'ATL':
            val = 12
        elif i['defteam'] == 'WAS':
            val = 13
        elif i['defteam'] == 'DAL':
            val = 14
        elif i['defteam'] == 'MIA':
            val = 15
        elif i['defteam'] == 'LAC':
            val = 16
        elif i['defteam'] == 'CHI':
            val = 17
        elif i['defteam'] == 'ARI':
            val = 18
        elif i['defteam'] == 'TEN':
            val = 19
        elif i['defteam'] == 'DEN':
            val = 20
        elif i['defteam'] == 'MIN':
            val = 21
        elif i['defteam'] == 'LAR':
            val = 22
        elif i['defteam'] == 'DET':
            val = 23
        elif i['defteam'] == 'NO':
            val = 24
        elif i['defteam'] == 'IND':
            val = 25
        elif i['defteam'] == 'BUF':
            val = 26
        elif i['defteam'] == 'NYG':
            val = 27
        elif i['defteam'] == 'CAR':
            val = 28
        elif i['defteam'] == 'LV':
            val = 29
        elif i['defteam'] == 'TB':
            val = 30
        elif i['defteam'] == 'NE':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2012:
        if i['defteam'] == 'PIT':
            val = 1
        elif i['defteam'] == 'DEN':
            val = 2
        elif i['defteam'] == 'SF':
            val = 3
        elif i['defteam'] == 'SEA':
            val = 4
        elif i['defteam'] == 'CHI':
            val = 5
        elif i['defteam'] == 'CIN':
            val = 6
        elif i['defteam'] == 'HOU':
            val = 7
        elif i['defteam'] == 'NYJ':
            val = 8
        elif i['defteam'] == 'LAC':
            val = 9
        elif i['defteam'] == 'CAR':
            val = 10
        elif i['defteam'] == 'GB':
            val = 11
        elif i['defteam'] == 'ARI':
            val = 12
        elif i['defteam'] == 'DET':
            val = 13
        elif i['defteam'] == 'LAR':
            val = 14
        elif i['defteam'] == 'PHI':
            val = 15
        elif i['defteam'] == 'MIN':
            val = 16
        elif i['defteam'] == 'BAL':
            val = 17
        elif i['defteam'] == 'LV':
            val = 18
        elif i['defteam'] == 'DAL':
            val = 19
        elif i['defteam'] == 'KC':
            val = 20
        elif i['defteam'] == 'MIA':
            val = 21
        elif i['defteam'] == 'BUF':
            val = 22
        elif i['defteam'] == 'CLE':
            val = 23
        elif i['defteam'] == 'ATL':
            val = 24
        elif i['defteam'] == 'NE':
            val = 25
        elif i['defteam'] == 'IND':
            val = 26
        elif i['defteam'] == 'TEN':
            val = 27
        elif i['defteam'] == 'WAS':
            val = 28
        elif i['defteam'] == 'TB':
            val = 29
        elif i['defteam'] == 'JAX':
            val = 30
        elif i['defteam'] == 'NYG':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2013:
        if i['defteam'] == 'SEA':
            val = 1
        elif i['defteam'] == 'CAR':
            val = 2
        elif i['defteam'] == 'CIN':
            val = 3
        elif i['defteam'] == 'NO':
            val = 4
        elif i['defteam'] == 'SF':
            val = 5
        elif i['defteam'] == 'ARI':
            val = 6
        elif i['defteam'] == 'HOU':
            val = 7
        elif i['defteam'] == 'NYG':
            val = 8
        elif i['defteam'] == 'CLE':
            val = 9
        elif i['defteam'] == 'BUF':
            val = 10
        elif i['defteam'] == 'NYJ':
            val = 11
        elif i['defteam'] == 'BAL':
            val = 12
        elif i['defteam'] == 'PIT':
            val = 13
        elif i['defteam'] == 'TEN':
            val = 14
        elif i['defteam'] == 'LAR':
            val = 15
        elif i['defteam'] == 'DET':
            val = 16
        elif i['defteam'] == 'TB':
            val = 17
        elif i['defteam'] == 'WAS':
            val = 18
        elif i['defteam'] == 'DEN':
            val = 19
        elif i['defteam'] == 'IND':
            val = 20
        elif i['defteam'] == 'MIA':
            val = 21
        elif i['defteam'] == 'LV':
            val = 22
        elif i['defteam'] == 'LAC':
            val = 23
        elif i['defteam'] == 'KC':
            val = 24
        elif i['defteam'] == 'GB':
            val = 25
        elif i['defteam'] == 'NE':
            val = 26
        elif i['defteam'] == 'ATL':
            val = 27
        elif i['defteam'] == 'JAX':
            val = 28
        elif i['defteam'] == 'PHI':
            val = 29
        elif i['defteam'] == 'CHI':
            val = 30
        elif i['defteam'] == 'MIN':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2014:
        if i['defteam'] == 'SEA':
            val = 1
        elif i['defteam'] == 'DET':
            val = 2
        elif i['defteam'] == 'DEN':
            val = 3
        elif i['defteam'] == 'BUF':
            val = 4
        elif i['defteam'] == 'SF':
            val = 5
        elif i['defteam'] == 'NYJ':
            val = 6
        elif i['defteam'] == 'KC':
            val = 7
        elif i['defteam'] == 'BAL':
            val = 8
        elif i['defteam'] == 'LAC':
            val = 9
        elif i['defteam'] == 'CAR':
            val = 10
        elif i['defteam'] == 'IND':
            val = 11
        elif i['defteam'] == 'MIA':
            val = 12
        elif i['defteam'] == 'NE':
            val = 13
        elif i['defteam'] == 'MIN':
            val = 14
        elif i['defteam'] == 'GB':
            val = 15
        elif i['defteam'] == 'HOU':
            val = 16
        elif i['defteam'] == 'LAR':
            val = 17
        elif i['defteam'] == 'PIT':
            val = 18
        elif i['defteam'] == 'DAL':
            val = 19
        elif i['defteam'] == 'WAS':
            val = 20
        elif i['defteam'] == 'LV':
            val = 21
        elif i['defteam'] == 'CIN':
            val = 22
        elif i['defteam'] == 'CLE':
            val = 23
        elif i['defteam'] == 'ARI':
            val = 24
        elif i['defteam'] == 'TB':
            val = 25
        elif i['defteam'] == 'JAX':
            val = 26
        elif i['defteam'] == 'TEN':
            val = 27
        elif i['defteam'] == 'PHI':
            val = 28
        elif i['defteam'] == 'NYG':
            val = 29
        elif i['defteam'] == 'CHI':
            val = 30
        elif i['defteam'] == 'NO':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2015:
        if i['defteam'] == 'DEN':
            val = 1
        elif i['defteam'] == 'SEA':
            val = 2
        elif i['defteam'] == 'HOU':
            val = 3
        elif i['defteam'] == 'NYJ':
            val = 4
        elif i['defteam'] == 'ARI':
            val = 5
        elif i['defteam'] == 'CAR':
            val = 6
        elif i['defteam'] == 'KC':
            val = 7
        elif i['defteam'] == 'BAL':
            val = 8
        elif i['defteam'] == 'NE':
            val = 9
        elif i['defteam'] == 'TB':
            val = 10
        elif i['defteam'] == 'CIN':
            val = 11
        elif i['defteam'] == 'TEN':
            val = 12
        elif i['defteam'] == 'MIN':
            val = 13
        elif i['defteam'] == 'CHI':
            val = 14
        elif i['defteam'] == 'GB':
            val = 15
        elif i['defteam'] == 'ATL':
            val = 16
        elif i['defteam'] == 'DAL':
            val = 17
        elif i['defteam'] == 'DET':
            val = 18
        elif i['defteam'] == 'BUF':
            val = 19
        elif i['defteam'] == 'LAC':
            val = 20
        elif i['defteam'] == 'PIT':
            val = 21
        elif i['defteam'] == 'LV':
            val = 22
        elif i['defteam'] == 'LAR':
            val = 23
        elif i['defteam'] == 'JAX':
            val = 24
        elif i['defteam'] == 'MIA':
            val = 25
        elif i['defteam'] == 'IND':
            val = 26
        elif i['defteam'] == 'CLE':
            val = 27
        elif i['defteam'] == 'WAS':
            val = 28
        elif i['defteam'] == 'SF':
            val = 29
        elif i['defteam'] == 'PHI':
            val = 30
        elif i['defteam'] == 'NO':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2016:
        if i['defteam'] == 'HOU':
            val = 1
        elif i['defteam'] == 'ARI':
            val = 2
        elif i['defteam'] == 'MIN':
            val = 3
        elif i['defteam'] == 'DEN':
            val = 4
        elif i['defteam'] == 'SEA':
            val = 5
        elif i['defteam'] == 'JAX':
            val = 6
        elif i['defteam'] == 'BAL':
            val = 7
        elif i['defteam'] == 'NE':
            val = 8
        elif i['defteam'] == 'LAR':
            val = 9
        elif i['defteam'] == 'NYG':
            val = 10
        elif i['defteam'] == 'NYJ':
            val = 11
        elif i['defteam'] == 'PIT':
            val = 12
        elif i['defteam'] == 'PHI':
            val = 13
        elif i['defteam'] == 'DAL':
            val = 14
        elif i['defteam'] == 'CHI':
            val = 15
        elif i['defteam'] == 'LAC':
            val = 16
        elif i['defteam'] == 'CIN':
            val = 17
        elif i['defteam'] == 'DET':
            val = 18
        elif i['defteam'] == 'BUF':
            val = 19
        elif i['defteam'] == 'TEN':
            val = 20
        elif i['defteam'] == 'CAR':
            val = 21
        elif i['defteam'] == 'GB':
            val = 22
        elif i['defteam'] == 'TB':
            val = 23
        elif i['defteam'] == 'KC':
            val = 24
        elif i['defteam'] == 'ATL':
            val = 25
        elif i['defteam'] == 'LV':
            val = 26
        elif i['defteam'] == 'NO':
            val = 27
        elif i['defteam'] == 'WAS':
            val = 28
        elif i['defteam'] == 'MIA':
            val = 29
        elif i['defteam'] == 'IND':
            val = 30
        elif i['defteam'] == 'CLE':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2017:
        if i['defteam'] == 'MIN':
            val = 1
        elif i['defteam'] == 'JAX':
            val = 2
        elif i['defteam'] == 'DEN':
            val = 3
        elif i['defteam'] == 'PHI':
            val = 4
        elif i['defteam'] == 'PIT':
            val = 5
        elif i['defteam'] == 'ARI':
            val = 6
        elif i['defteam'] == 'CAR':
            val = 7
        elif i['defteam'] == 'DAL':
            val = 8
        elif i['defteam'] == 'ATL':
            val = 9
        elif i['defteam'] == 'CHI':
            val = 10
        elif i['defteam'] == 'SEA':
            val = 11
        elif i['defteam'] == 'BAL':
            val = 12
        elif i['defteam'] == 'TEN':
            val = 13
        elif i['defteam'] == 'CLE':
            val = 14
        elif i['defteam'] == 'LAC':
            val = 15
        elif i['defteam'] == 'MIA':
            val = 16
        elif i['defteam'] == 'NO':
            val = 17
        elif i['defteam'] == 'CIN':
            val = 18
        elif i['defteam'] == 'LAR':
            val = 19
        elif i['defteam'] == 'HOU':
            val = 20
        elif i['defteam'] == 'WAS':
            val = 21
        elif i['defteam'] == 'GB':
            val = 22
        elif i['defteam'] == 'LV':
            val = 23
        elif i['defteam'] == 'SF':
            val = 24
        elif i['defteam'] == 'NYJ':
            val = 25
        elif i['defteam'] == 'BUF':
            val = 26
        elif i['defteam'] == 'DET':
            val = 27
        elif i['defteam'] == 'KC':
            val = 28
        elif i['defteam'] == 'NE':
            val = 29
        elif i['defteam'] == 'IND':
            val = 30
        elif i['defteam'] == 'NYG':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2018:
        if i['defteam'] == 'BAL':
            val = 1
        elif i['defteam'] == 'BUF':
            val = 2
        elif i['defteam'] == 'CHI':
            val = 3
        elif i['defteam'] == 'MIN':
            val = 4
        elif i['defteam'] == 'JAX':
            val = 5
        elif i['defteam'] == 'PIT':
            val = 6
        elif i['defteam'] == 'DAL':
            val = 7
        elif i['defteam'] == 'TEN':
            val = 8
        elif i['defteam'] == 'LAC':
            val = 9
        elif i['defteam'] == 'DET':
            val = 10
        elif i['defteam'] == 'IND':
            val = 11
        elif i['defteam'] == 'HOU':
            val = 12
        elif i['defteam'] == 'SF':
            val = 13
        elif i['defteam'] == 'NO':
            val = 14
        elif i['defteam'] == 'CAR':
            val = 15
        elif i['defteam'] == 'SEA':
            val = 16
        elif i['defteam'] == 'WAS':
            val = 17
        elif i['defteam'] == 'GB':
            val = 18
        elif i['defteam'] == 'LAR':
            val = 19
        elif i['defteam'] == 'ARI':
            val = 20
        elif i['defteam'] == 'NE':
            val = 21
        elif i['defteam'] == 'DEN':
            val = 22
        elif i['defteam'] == 'PHI':
            val = 23
        elif i['defteam'] == 'NYG':
            val = 24
        elif i['defteam'] == 'NYJ':
            val = 25
        elif i['defteam'] == 'LV':
            val = 26
        elif i['defteam'] == 'TB':
            val = 27
        elif i['defteam'] == 'ATL':
            val = 28
        elif i['defteam'] == 'MIA':
            val = 29
        elif i['defteam'] == 'CLE':
            val = 30
        elif i['defteam'] == 'KC':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2019:
        if i['defteam'] == 'NE':
            val = 1
        elif i['defteam'] == 'SF':
            val = 2
        elif i['defteam'] == 'BUF':
            val = 3
        elif i['defteam'] == 'BAL':
            val = 4
        elif i['defteam'] == 'PIT':
            val = 5
        elif i['defteam'] == 'LAC':
            val = 6
        elif i['defteam'] == 'NYJ':
            val = 7
        elif i['defteam'] == 'CHI':
            val = 8
        elif i['defteam'] == 'DAL':
            val = 9
        elif i['defteam'] == 'PHI':
            val = 10
        elif i['defteam'] == 'NO':
            val = 11
        elif i['defteam'] == 'LAR':
            val = 12
        elif i['defteam'] == 'MIN':
            val = 13
        elif i['defteam'] == 'TB':
            val = 14
        elif i['defteam'] == 'IND':
            val = 15
        elif i['defteam'] == 'KC':
            val = 16
        elif i['defteam'] == 'GB':
            val = 17
        elif i['defteam'] == 'LV':
            val = 18
        elif i['defteam'] == 'ATL':
            val = 19
        elif i['defteam'] == 'TEN':
            val = 20
        elif i['defteam'] == 'CLE':
            val = 21
        elif i['defteam'] == 'CAR':
            val = 22
        elif i['defteam'] == 'JAX':
            val = 23
        elif i['defteam'] == 'NYG':
            val = 24
        elif i['defteam'] == 'SEA':
            val = 25
        elif i['defteam'] == 'WAS':
            val = 26
        elif i['defteam'] == 'HOU':
            val = 27
        elif i['defteam'] == 'CIN':
            val = 28
        elif i['defteam'] == 'MIA':
            val = 29
        elif i['defteam'] == 'IND':
            val = 30
        elif i['defteam'] == 'DET':
            val = 31
        else:
            val = 32
        return val
    elif i['season'] == 2020:
        if i['defteam'] == 'LAR':
            val = 1
        elif i['defteam'] == 'WAS':
            val = 2
        elif i['defteam'] == 'PIT':
            val = 3
        elif i['defteam'] == 'NO':
            val = 4
        elif i['defteam'] == 'SF':
            val = 5
        elif i['defteam'] == 'TB':
            val = 6
        elif i['defteam'] == 'BAL':
            val = 7
        elif i['defteam'] == 'IND':
            val = 8
        elif i['defteam'] == 'GB':
            val = 9
        elif i['defteam'] == 'LAC':
            val = 10
        elif i['defteam'] == 'CHI':
            val = 11
        elif i['defteam'] == 'NYG':
            val = 12
        elif i['defteam'] == 'ARI':
            val = 13
        elif i['defteam'] == 'BUF':
            val = 14
        elif i['defteam'] == 'NE':
            val = 15
        elif i['defteam'] == 'KC':
            val = 16
        elif i['defteam'] == 'CLE':
            val = 17
        elif i['defteam'] == 'CAR':
            val = 18
        elif i['defteam'] == 'PHI':
            val = 19
        elif i['defteam'] == 'MIA':
            val = 20
        elif i['defteam'] == 'DEN':
            val = 21
        elif i['defteam'] == 'SEA':
            val = 22
        elif i['defteam'] == 'DAL':
            val = 23
        elif i['defteam'] == 'NYJ':
            val = 24
        elif i['defteam'] == 'LV':
            val = 25
        elif i['defteam'] == 'CIN':
            val = 26
        elif i['defteam'] == 'MIN':
            val = 27
        elif i['defteam'] == 'TEN':
            val = 28
        elif i['defteam'] == 'ATL':
            val = 29
        elif i['defteam'] == 'HOU':
            val = 30
        elif i['defteam'] == 'JAX':
            val = 31
        else:
            val = 32
        return val
    else:
        return

In [21]:
df['def_rank'] = df.apply(def_rank, axis=1)

In [22]:
df['def_rank'].head(100)

49      13
181     13
208     32
323     32
511      7
        ..
7865    20
7992    24
8045     8
8146     5
8173     5
Name: def_rank, Length: 100, dtype: int64

In [23]:
df['def_rank'].unique()

array([13, 32,  7, 23, 27,  9, 30, 16, 31, 26, 25, 20, 24,  2, 19,  3, 12,
       22, 29,  4,  6, 28, 21, 14, 17, 10, 11,  5,  8, 18, 15,  1],
      dtype=int64)

## EDA Visualizations

# The Model

### Drop columns that are no longer needed

In [24]:
df['surface'].value_counts()

grass         2797
fieldturf     1141
sportturf      312
matrixturf     135
astroturf      124
a_turf         118
astroplay       67
fieldturf       33
Name: surface, dtype: int64

In [25]:
df = df.drop(['season', 'week', 'home_team', 'away_team', 'posteam', 'defteam', 'yards_gained', 'timeout_team', 
              'score_differential_post', 'no_score_prob', 'epa', 'wpa', 'complete_pass', 'incomplete_pass', 'interception', 
              'tackled_for_loss', 'fumble_lost', 'qb_hit', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'fumble', 
              'stadium'], axis=1)

In [26]:
df.columns

Index(['posteam_type', 'yardline_100', 'half_seconds_remaining', 'game_half',
       'drive', 'qtr', 'down', 'goal_to_go', 'ydstogo', 'ydsnet', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_scramble', 'pass_length',
       'pass_location', 'air_yards', 'yards_after_catch', 'run_location',
       'run_gap', 'timeout', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'score_differential', 'opp_td_prob',
       'td_prob', 'ep', 'wp', 'series', 'drive_play_count', 'drive_inside20',
       'div_game', 'surface', 'roof', 'pass', 'rush', 'success', 'off_rank',
       'def_rank'],
      dtype='object')

In [27]:
df.head()

,posteam_type,yardline_100,half_seconds_remaining,game_half,drive,qtr,down,goal_to_go,ydstogo,ydsnet,...,drive_play_count,drive_inside20,div_game,surface,roof,pass,rush,success,off_rank,def_rank
49,home,1.0,757.0,Half1,9.0,2,4.0,1,1,61.0,...,10.0,1.0,1,grass,outdoors,0.0,1.0,0.0,15,13
181,home,16.0,37.0,Half2,22.0,4,4.0,0,5,59.0,...,12.0,1.0,1,grass,outdoors,1.0,0.0,0.0,15,13
208,away,34.0,1442.0,Half1,3.0,1,4.0,0,7,41.0,...,8.0,0.0,1,fieldturf,dome,1.0,0.0,0.0,26,32
323,away,10.0,900.0,Half2,17.0,4,4.0,0,1,59.0,...,11.0,1.0,1,fieldturf,dome,0.0,1.0,1.0,26,32
511,away,28.0,1088.0,Half2,15.0,3,4.0,0,4,33.0,...,10.0,0.0,1,grass,outdoors,1.0,0.0,0.0,16,7


## Build the Model

### Create dummy variables to account for categorical values

In [28]:
df.dtypes

posteam_type                   object
yardline_100                  float64
half_seconds_remaining        float64
game_half                      object
drive                         float64
qtr                             int64
down                          float64
goal_to_go                      int64
ydstogo                         int64
ydsnet                        float64
shotgun                         int64
no_huddle                       int64
qb_dropback                   float64
qb_scramble                     int64
pass_length                    object
pass_location                  object
air_yards                     float64
yards_after_catch             float64
run_location                   object
run_gap                        object
timeout                       float64
posteam_timeouts_remaining    float64
defteam_timeouts_remaining    float64
score_differential            float64
opp_td_prob                   float64
td_prob                       float64
ep          

In [29]:
dummy_postype = pd.get_dummies(df['posteam_type'])
dummy_postype.head()

,away,home
49,0,1
181,0,1
208,1,0
323,1,0
511,1,0


In [30]:
dummy_half = pd.get_dummies(df['game_half'])
dummy_half.head()

,Half1,Half2,Overtime
49,1,0,0
181,0,1,0
208,1,0,0
323,0,1,0
511,0,1,0


In [31]:
dummy_pass_length = pd.get_dummies(df['pass_length'])
dummy_pass_length.head()

,deep,short
49,0,0
181,0,1
208,1,0
323,0,0
511,0,0


In [32]:
dummy_pass_loc = pd.get_dummies(df['pass_location'])
dummy_pass_loc.head()

,left,middle,right
49,0,0,0
181,0,0,1
208,0,1,0
323,0,0,0
511,0,0,0


In [33]:
dummy_run_loc = pd.get_dummies(df['run_location'])
dummy_run_loc.head()

,left,middle,right
49,0,1,0
181,0,0,0
208,0,0,0
323,1,0,0
511,0,0,0


In [34]:
dummy_run_gap = pd.get_dummies(df['run_gap'])
dummy_run_gap.head()

,end,guard,tackle
49,0,0,0
181,0,0,0
208,0,0,0
323,1,0,0
511,0,0,0


In [35]:
dummy_surface = pd.get_dummies(df['surface'])
dummy_surface.head(10)

,a_turf,astroplay,astroturf,fieldturf,fieldturf,grass,matrixturf,sportturf
49,0,0,0,0,0,1,0,0
181,0,0,0,0,0,1,0,0
208,0,0,0,1,0,0,0,0
323,0,0,0,1,0,0,0,0
511,0,0,0,0,0,1,0,0
533,0,0,0,0,0,1,0,0
584,0,0,0,0,0,0,1,0
708,0,0,0,0,0,0,1,0
737,0,0,0,0,0,0,1,0
743,0,0,0,0,0,0,1,0


In [36]:
dummy_roof = pd.get_dummies(df['roof'])
dummy_roof.head()

,closed,dome,open,outdoors
49,0,0,0,1
181,0,0,0,1
208,0,1,0,0
323,0,1,0,0
511,0,0,0,1


In [37]:
df = df.merge(dummy_postype, left_index=True, right_index=True)
df = df.merge(dummy_half, left_index=True, right_index=True)
df = df.merge(dummy_pass_length, left_index=True, right_index=True)
df = df.merge(dummy_pass_loc, left_index=True, right_index=True)
df = df.merge(dummy_run_loc, left_index=True, right_index=True)
df = df.merge(dummy_run_gap, left_index=True, right_index=True)
df = df.merge(dummy_surface, left_index=True, right_index=True)
df = df.merge(dummy_roof, left_index=True, right_index=True)
df.head()

,posteam_type,yardline_100,half_seconds_remaining,game_half,drive,qtr,down,goal_to_go,ydstogo,ydsnet,...,astroturf,fieldturf,fieldturf,grass,matrixturf,sportturf,closed,dome,open,outdoors
49,home,1.0,757.0,Half1,9.0,2,4.0,1,1,61.0,...,0,0,0,1,0,0,0,0,0,1
181,home,16.0,37.0,Half2,22.0,4,4.0,0,5,59.0,...,0,0,0,1,0,0,0,0,0,1
208,away,34.0,1442.0,Half1,3.0,1,4.0,0,7,41.0,...,0,1,0,0,0,0,0,1,0,0
323,away,10.0,900.0,Half2,17.0,4,4.0,0,1,59.0,...,0,1,0,0,0,0,0,1,0,0
511,away,28.0,1088.0,Half2,15.0,3,4.0,0,4,33.0,...,0,0,0,1,0,0,0,0,0,1


In [38]:
df = df.drop(['posteam_type', 'game_half', 'pass_length', 'pass_location', 'run_location', 'run_gap', 'surface', 'roof'], axis=1)
df.dtypes

yardline_100                  float64
half_seconds_remaining        float64
drive                         float64
qtr                             int64
down                          float64
goal_to_go                      int64
ydstogo                         int64
ydsnet                        float64
shotgun                         int64
no_huddle                       int64
qb_dropback                   float64
qb_scramble                     int64
air_yards                     float64
yards_after_catch             float64
timeout                       float64
posteam_timeouts_remaining    float64
defteam_timeouts_remaining    float64
score_differential            float64
opp_td_prob                   float64
td_prob                       float64
ep                            float64
wp                            float64
series                          int64
drive_play_count              float64
drive_inside20                float64
div_game                        int64
pass        

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4727 entries, 49 to 481749
Data columns (total 59 columns):
yardline_100                  4727 non-null float64
half_seconds_remaining        4727 non-null float64
drive                         4727 non-null float64
qtr                           4727 non-null int64
down                          4727 non-null float64
goal_to_go                    4727 non-null int64
ydstogo                       4727 non-null int64
ydsnet                        4727 non-null float64
shotgun                       4727 non-null int64
no_huddle                     4727 non-null int64
qb_dropback                   4716 non-null float64
qb_scramble                   4727 non-null int64
air_yards                     2882 non-null float64
yards_after_catch             1511 non-null float64
timeout                       4716 non-null float64
posteam_timeouts_remaining    4727 non-null float64
defteam_timeouts_remaining    4727 non-null float64
score_differential

### Find and replace/drop null values

In [40]:
# null values in these columns mean the value should be zero
df[['qb_dropback', 'air_yards', 'yards_after_catch', 'timeout', 
    'posteam_timeouts_remaining', 'defteam_timeouts_remaining']] = df[['qb_dropback', 'air_yards', 'yards_after_catch', 
                                                                       'timeout', 'posteam_timeouts_remaining', 
                                                                       'defteam_timeouts_remaining']].fillna(0)

In [41]:
# remaining values are unknown
# mean, median and mode will not work
# rows should be dropped
df = df.dropna()

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4702 entries, 49 to 481749
Data columns (total 59 columns):
yardline_100                  4702 non-null float64
half_seconds_remaining        4702 non-null float64
drive                         4702 non-null float64
qtr                           4702 non-null int64
down                          4702 non-null float64
goal_to_go                    4702 non-null int64
ydstogo                       4702 non-null int64
ydsnet                        4702 non-null float64
shotgun                       4702 non-null int64
no_huddle                     4702 non-null int64
qb_dropback                   4702 non-null float64
qb_scramble                   4702 non-null int64
air_yards                     4702 non-null float64
yards_after_catch             4702 non-null float64
timeout                       4702 non-null float64
posteam_timeouts_remaining    4702 non-null float64
defteam_timeouts_remaining    4702 non-null float64
score_differential

## Build the Baseline with a Decision Tree

### Import Libraries and Test Data

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [44]:
#create numpy arrays for predictors and target variables 
X = df.drop('success', axis=1).values
y = df['success'].values

In [45]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [46]:
# Instantiate and fit a DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(criterion='gini', max_depth=5) 
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [47]:
# Feature importance
tree_clf.feature_importances_

array([0.15648674, 0.00731583, 0.        , 0.        , 0.        ,
       0.        , 0.03480968, 0.41300809, 0.        , 0.        ,
       0.        , 0.02179632, 0.01533823, 0.10732105, 0.        ,
       0.        , 0.        , 0.        , 0.00999819, 0.03511177,
       0.        , 0.00793557, 0.        , 0.07145151, 0.03151908,
       0.        , 0.        , 0.08790794, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [48]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [49]:
print('-'*40)
print('Accuracy Score:')
print(accuracy_score(y_test, y_pred))

print('-'*40)
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

print('-'*40)
print('Classification Matrix:')
print(classification_report(y_test, y_pred))

----------------------------------------
Accuracy Score:
0.8164422395464209
----------------------------------------
Confusion Matrix:
[[570 140]
 [119 582]]
----------------------------------------
Classification Matrix:
              precision    recall  f1-score   support

         0.0       0.83      0.80      0.81       710
         1.0       0.81      0.83      0.82       701

    accuracy                           0.82      1411
   macro avg       0.82      0.82      0.82      1411
weighted avg       0.82      0.82      0.82      1411



In [50]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp

(570, 140, 119, 582)

In [51]:
print("Testing Accuracy for Decision Tree Classifier: {:.4}%".format(accuracy_score(y_test, y_pred) * 100))

Testing Accuracy for Decision Tree Classifier: 81.64%
